### This notebook is Final submission for the hackathon 

https://www.machinehack.com/hackathons/predicting_food_delivery_time_hackathon_by_ims_proschool/data

Alert: This notebook is an old notebook so I couldn't structure the preprocessing and model code.

In [1]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
train_data = pd.read_excel("Data_Train.xlsx")

In [3]:
test_data =pd.read_excel("Data_Test.xlsx")

In [4]:
train_data.head(5)

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
0,ID_6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",₹200,₹50,3.5,12,4,30 minutes
1,ID_2882,"Sector 3, Marathalli","Ice Cream, Desserts",₹100,₹50,3.5,11,4,30 minutes
2,ID_1595,Mumbai Central,"Italian, Street Food, Fast Food",₹150,₹50,3.6,99,30,65 minutes
3,ID_5929,"Sector 1, Noida","Mughlai, North Indian, Chinese",₹250,₹99,3.7,176,95,30 minutes
4,ID_6123,"Rmz Centennial, I Gate, Whitefield","Cafe, Beverages",₹200,₹99,3.2,521,235,65 minutes


In [5]:
test_data.head(5)

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews
0,ID_2842,"Mico Layout, Stage 2, BTM Layout,Bangalore","North Indian, Chinese, Assamese",₹350,₹50,4.2,361,225
1,ID_730,"Mico Layout, Stage 2, BTM Layout,Bangalore","Biryani, Kebab",₹100,₹50,NEW,-,-
2,ID_4620,"Sector 1, Noida",Fast Food,₹100,₹50,3.6,36,16
3,ID_5470,"Babarpur, New Delhi, Delhi","Mithai, North Indian, Chinese, Fast Food, Sout...",₹200,₹50,3.6,66,33
4,ID_3249,"Sector 1, Noida","Chinese, Fast Food",₹150,₹50,2.9,38,14


In [6]:
test_data.shape

(2774, 8)

In [7]:
train_data.shape

(11094, 9)

In [8]:
print(set(test_data.Location.unique())- set(train_data.Location.unique()))
print(set(train_data.Location.unique())- set(test_data.Location.unique()))

set()
set()


In [9]:
test_data.Location.nunique()

35

In [10]:
train_data.Location.nunique()

35

In [11]:
train_data.Restaurant.nunique()

7480

In [12]:
test_data.Restaurant.nunique()

2401

In [13]:
print(len(set(train_data.Restaurant.unique())-set(test_data.Restaurant.unique())))
print(len(set(test_data.Restaurant.unique())-set(train_data.Restaurant.unique())))

6260
1181


In [14]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2774 entries, 0 to 2773
Data columns (total 8 columns):
Restaurant       2774 non-null object
Location         2774 non-null object
Cuisines         2774 non-null object
Average_Cost     2774 non-null object
Minimum_Order    2774 non-null object
Rating           2774 non-null object
Votes            2774 non-null object
Reviews          2774 non-null object
dtypes: object(8)
memory usage: 173.5+ KB


In [15]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11094 entries, 0 to 11093
Data columns (total 9 columns):
Restaurant       11094 non-null object
Location         11094 non-null object
Cuisines         11094 non-null object
Average_Cost     11094 non-null object
Minimum_Order    11094 non-null object
Rating           11094 non-null object
Votes            11094 non-null object
Reviews          11094 non-null object
Delivery_Time    11094 non-null object
dtypes: object(9)
memory usage: 780.1+ KB


In [16]:
train_data.loc[train_data['Reviews'].str.contains("-")]

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
12,ID_1156,"Sector 1, Noida","Burger, Fast Food, Desserts, Beverages",₹150,₹50,-,-,-,30 minutes
16,ID_2141,"Sector 1, Noida",Italian,₹100,₹50,-,-,-,30 minutes
18,ID_5262,Pune University,"Kerala, Bihari",₹150,₹50,-,-,-,30 minutes
25,ID_4944,"Mico Layout, Stage 2, BTM Layout,Bangalore","Desserts, Ice Cream",₹100,₹50,-,-,-,45 minutes
47,ID_2809,"Mico Layout, Stage 2, BTM Layout,Bangalore",Chinese,₹200,₹50,NEW,-,-,30 minutes
49,ID_4866,Delhi University-GTB Nagar,Lucknowi,₹200,₹50,-,-,-,30 minutes
61,ID_294,"Sector 1, Noida","Bakery, Desserts, Fast Food",₹100,₹50,NEW,-,-,30 minutes
71,ID_3309,"D-Block, Sector 63, Noida","Chinese, Rolls",₹150,₹50,-,-,-,30 minutes
75,ID_3087,"Chandni Chowk, Kolkata","Fast Food, Bengali",₹100,₹50,-,-,-,30 minutes
79,ID_6205,"Sector 14, Noida",Rolls,₹100,₹50,NEW,-,-,30 minutes


In [17]:
test_data.loc[test_data['Reviews'].str.contains("-")]

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews
1,ID_730,"Mico Layout, Stage 2, BTM Layout,Bangalore","Biryani, Kebab",₹100,₹50,NEW,-,-
8,ID_7982,"Sector 1, Noida",North Indian,₹200,₹50,NEW,-,-
11,ID_3977,"D-Block, Sector 63, Noida",North Indian,₹200,₹50,NEW,-,-
17,ID_2538,"Laxman Vihar Industrial Area, Sector 3A, Gurgoan","Fast Food, North Indian",₹200,₹50,-,-,-
20,ID_1977,"Sector 1, Noida","North Indian, Chinese",₹150,₹50,NEW,-,-
26,ID_4175,"FTI College, Law College Road, Pune",South Indian,₹100,₹50,-,-,-
29,ID_1542,"D-Block, Sector 63, Noida",North Indian,₹250,₹50,-,-,-
47,ID_1875,"Mico Layout, Stage 2, BTM Layout,Bangalore","North Indian, Chinese",₹200,₹50,NEW,-,-
48,ID_4906,"Sector 63A,Gurgaon","North Indian, Chinese, Biryani",₹200,₹50,NEW,-,-
52,ID_6944,Majestic,"Fast Food, Beverages",₹100,₹50,NEW,-,-


In [18]:
all_data = train_data.drop("Delivery_Time",axis=1).append(test_data,ignore_index=True)

In [19]:
all_data[:11094].tail(5)

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews
11089,ID_8067,"BTM Layout 1, Electronic City","Tibetan, Chinese, Continental, Momos",₹250,₹50,4.2,326,189
11090,ID_4620,"Sector 14, Noida",Fast Food,₹100,₹50,3.6,36,16
11091,ID_3392,Majestic,"South Indian, Chinese, North Indian",₹100,₹50,3.5,45,18
11092,ID_4115,"Sector 3, Marathalli",North Indian,₹100,₹50,3.1,24,9
11093,ID_4417,"Sector 63A,Gurgaon",North Indian,₹100,₹50,NEW,-,-


In [20]:
train_data.tail(5)

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
11089,ID_8067,"BTM Layout 1, Electronic City","Tibetan, Chinese, Continental, Momos",₹250,₹50,4.2,326,189,30 minutes
11090,ID_4620,"Sector 14, Noida",Fast Food,₹100,₹50,3.6,36,16,30 minutes
11091,ID_3392,Majestic,"South Indian, Chinese, North Indian",₹100,₹50,3.5,45,18,30 minutes
11092,ID_4115,"Sector 3, Marathalli",North Indian,₹100,₹50,3.1,24,9,30 minutes
11093,ID_4417,"Sector 63A,Gurgaon",North Indian,₹100,₹50,NEW,-,-,30 minutes


In [21]:
all_data[11094:].head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews
11094,ID_2842,"Mico Layout, Stage 2, BTM Layout,Bangalore","North Indian, Chinese, Assamese",₹350,₹50,4.2,361,225
11095,ID_730,"Mico Layout, Stage 2, BTM Layout,Bangalore","Biryani, Kebab",₹100,₹50,NEW,-,-
11096,ID_4620,"Sector 1, Noida",Fast Food,₹100,₹50,3.6,36,16
11097,ID_5470,"Babarpur, New Delhi, Delhi","Mithai, North Indian, Chinese, Fast Food, Sout...",₹200,₹50,3.6,66,33
11098,ID_3249,"Sector 1, Noida","Chinese, Fast Food",₹150,₹50,2.9,38,14


In [22]:
test_data.head(5)

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews
0,ID_2842,"Mico Layout, Stage 2, BTM Layout,Bangalore","North Indian, Chinese, Assamese",₹350,₹50,4.2,361,225
1,ID_730,"Mico Layout, Stage 2, BTM Layout,Bangalore","Biryani, Kebab",₹100,₹50,NEW,-,-
2,ID_4620,"Sector 1, Noida",Fast Food,₹100,₹50,3.6,36,16
3,ID_5470,"Babarpur, New Delhi, Delhi","Mithai, North Indian, Chinese, Fast Food, Sout...",₹200,₹50,3.6,66,33
4,ID_3249,"Sector 1, Noida","Chinese, Fast Food",₹150,₹50,2.9,38,14


In [23]:
all_data['Cuisines'] = all_data['Cuisines'].apply(lambda x:x.split(','))

In [24]:
all_data['Cuisines']= all_data['Cuisines'].apply(lambda x :[i.strip() for i in x] )

In [25]:
location = all_data['Location'].apply(lambda x:x.split(',')).apply(lambda x :[i.strip() for i in x])

In [26]:
mlb = MultiLabelBinarizer()
s= all_data['Cuisines']
cuisines = pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_)

In [27]:
mlb1 = MultiLabelBinarizer()
location_encoded = pd.DataFrame(mlb1.fit_transform(location),columns=mlb1.classes_)

In [28]:
cuisines['Chinese'].sum()

4339

In [29]:
pd.get_dummies(pd.DataFrame(all_data.Cuisines.values.tolist()),prefix_sep='',prefix='').sum(level=0,axis=1)['Chinese'].sum()

4339

In [30]:
train_data[train_data['Cuisines'].str.contains('Chinese')].count()

Restaurant       3459
Location         3459
Cuisines         3459
Average_Cost     3459
Minimum_Order    3459
Rating           3459
Votes            3459
Reviews          3459
Delivery_Time    3459
dtype: int64

In [31]:
location_encoded.head()

,BTM Layout,BTM Layout 1,Babarpur,Bangalore,Begumpet,Chandni Chowk,Chatta Bazaar,D-Block,Delhi,Delhi Administration Flats,...,Sector 3A,Sector 63,Sector 63A,Stage 2,Tejas Nagar Colony,Timarpur,Tiretti,Wadala West,Whitefield,Yerawada
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [32]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13868 entries, 0 to 13867
Data columns (total 8 columns):
Restaurant       13868 non-null object
Location         13868 non-null object
Cuisines         13868 non-null object
Average_Cost     13868 non-null object
Minimum_Order    13868 non-null object
Rating           13868 non-null object
Votes            13868 non-null object
Reviews          13868 non-null object
dtypes: object(8)
memory usage: 866.8+ KB


In [33]:
all_data.loc[~all_data['Average_Cost'].str.contains('₹'),'Average_Cost']='₹50'

In [34]:
all_data.loc[~all_data['Minimum_Order'].str.contains('₹')]

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews


In [35]:
all_data['Average_Cost'] = all_data['Average_Cost'].str.replace('₹','').str.replace(',','').astype(int)

In [36]:
all_data['Minimum_Order'] = all_data['Minimum_Order'].str.replace('₹','').str.replace(',','').astype(int)

In [37]:
all_data.Reviews.value_counts()

-       2905
2        551
1        483
3        466
4        428
5        320
7        264
6        260
8        242
9        239
10       223
11       193
12       169
14       158
13       157
23       141
15       138
18       136
17       131
16       128
24       113
30       111
20       109
22        99
19        92
26        90
21        88
31        86
25        86
32        86
        ... 
452        1
752        1
566        1
392        1
373        1
815        1
1225       1
959        1
395        1
477        1
478        1
1028       1
1636       1
774        1
419        1
800        1
1164       1
354        1
459        1
719        1
1093       1
1412       1
397        1
1439       1
600        1
726        1
772        1
1357       1
343        1
1734       1
Name: Reviews, Length: 811, dtype: int64

In [38]:
all_data.Reviews = all_data.Reviews.str.replace('-','0').astype(int)

In [39]:
all_data.Votes = all_data.Votes.str.replace('-','0').astype(int)

In [40]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13868 entries, 0 to 13867
Data columns (total 8 columns):
Restaurant       13868 non-null object
Location         13868 non-null object
Cuisines         13868 non-null object
Average_Cost     13868 non-null int64
Minimum_Order    13868 non-null int64
Rating           13868 non-null object
Votes            13868 non-null int64
Reviews          13868 non-null int64
dtypes: int64(4), object(4)
memory usage: 866.8+ KB


In [41]:
all_data.Rating.value_counts()

-                     1496
3.7                   1081
3.6                   1069
3.5                   1015
3.8                    983
3.9                    965
NEW                    958
3.4                    903
3.3                    828
4.0                    755
3.2                    640
4.1                    574
3.1                    531
3.0                    367
4.2                    342
4.3                    299
2.9                    256
2.8                    198
4.4                    171
2.7                     98
4.5                     96
2.6                     51
4.6                     48
4.7                     42
2.5                     33
2.4                     18
4.8                     16
Opening Soon            14
2.3                      8
4.9                      8
2.1                      2
Temporarily Closed       2
2.2                      1
Name: Rating, dtype: int64

In [42]:
all_data.Rating = all_data.Rating.str.replace('-','0.0').str.replace('Opening Soon','0.0'
                                                  ).str.replace('Temporarily Closed','0.0'
                                                               ).str.replace('NEW','0.0')

In [43]:
all_data.Rating = all_data.Rating.astype(float)

In [44]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13868 entries, 0 to 13867
Data columns (total 8 columns):
Restaurant       13868 non-null object
Location         13868 non-null object
Cuisines         13868 non-null object
Average_Cost     13868 non-null int64
Minimum_Order    13868 non-null int64
Rating           13868 non-null float64
Votes            13868 non-null int64
Reviews          13868 non-null int64
dtypes: float64(1), int64(4), object(3)
memory usage: 866.8+ KB


In [45]:
all_data_encoded = pd.concat([pd.concat([all_data,location_encoded],axis=1),cuisines],axis=1)

In [46]:
all_data_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13868 entries, 0 to 13867
Columns: 174 entries, Restaurant to Wraps
dtypes: float64(1), int64(170), object(3)
memory usage: 18.4+ MB


In [47]:
all_data_restraunt_encoded = pd.concat([all_data_encoded,pd.get_dummies(all_data_encoded.Restaurant)],axis=1)

In [48]:
all_data_encoded = all_data_encoded.drop(['Restaurant','Location','Cuisines'],axis=1)

In [49]:
all_data_restraunt_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13868 entries, 0 to 13867
Columns: 8835 entries, Restaurant to ID_999
dtypes: float64(1), int64(170), object(3), uint8(8661)
memory usage: 133.0+ MB


In [50]:
all_data_restraunt_encoded = all_data_restraunt_encoded.drop(['Restaurant','Location','Cuisines'],axis=1)

In [51]:
pd.read_excel("Sample_Submission.xlsx").head(5)

,Delivery_Time
0,120 minutes
1,20 minutes
2,20 minutes
3,65 minutes
4,45 minutes


In [52]:
train_data['Delivery_Time'].value_counts()

30 minutes     7406
45 minutes     2665
65 minutes      923
120 minutes      62
20 minutes       20
80 minutes       14
10 minutes        4
Name: Delivery_Time, dtype: int64

In [53]:
#target = train_data['Delivery_Time']

In [54]:
factor = pd.factorize(train_data['Delivery_Time'])
definitions = factor[1]
target = pd.Series(factor[0])


In [55]:
all_data_encoded['cuisines_count']=all_data.Cuisines.apply(lambda x:len(x))

In [56]:
all_data_encoded.to_csv('All_data_encoded.csv',index=False)

## Model Building 

In [58]:
from sklearn.model_selection import KFold,cross_val_score
from sklearn.ensemble import RandomForestClassifier


In [59]:
X_train, X_test, y_train, y_test = train_test_split(all_data_encoded[0:11094], target, test_size = 0.25, random_state = 21)

In [60]:
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [61]:
y_submission = classifier.predict(all_data_encoded[11094:])

In [225]:
pd.DataFrame({'Delivery_Time':np.vectorize(reversefactor.get)(y_submission)}).to_excel(
    "Submissions/First_submission_rfc.xlsx",index=False)

In [62]:
def plot_confusion_matrix(classifier,X,y):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 21)
    classifier.fit(X_train, y_train)
    # Predicting the Test set results
    y_pred = classifier.predict(X_test)
    #Reverse factorize (converting y_pred from 0s,1s and 2s to Iris-setosa, Iris-versicolor and Iris-virginica
    reversefactor = dict(zip(range(7),definitions))
    y_test = np.vectorize(reversefactor.get)(y_test)
    y_pred = np.vectorize(reversefactor.get)(y_pred)
    # Making the Confusion Matrix
    print(pd.crosstab(y_test, y_pred, rownames=['Actual duration'], colnames=['Predicted duration']))

In [63]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
#Reverse factorize (converting y_pred from 0s,1s and 2s to Iris-setosa, Iris-versicolor and Iris-virginica
reversefactor = dict(zip(range(7),definitions))
y_test = np.vectorize(reversefactor.get)(y_test)
y_pred = np.vectorize(reversefactor.get)(y_pred)
# Making the Confusion Matrix
print(pd.crosstab(y_test, y_pred, rownames=['Actual duration'], colnames=['Predicted duration']))

Predicted duration  120 minutes  30 minutes  45 minutes  65 minutes  \
Actual duration                                                       
10 minutes                    0           1           0           0   
120 minutes                   3           3           2           2   
20 minutes                    0           6           1           0   
30 minutes                    1        1672         143          29   
45 minutes                    0         334         293          30   
65 minutes                    0         110          74          63   
80 minutes                    0           3           3           0   

Predicted duration  80 minutes  
Actual duration                 
10 minutes                   0  
120 minutes                  0  
20 minutes                   0  
30 minutes                   0  
45 minutes                   0  
65 minutes                   0  
80 minutes                   1  


In [66]:
import lightgbm as lgbm

/Users/lavanyamk/.local/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [64]:
from sklearn.ensemble import AdaBoostClassifier,ExtraTreesClassifier,BaggingClassifier,GradientBoostingClassifier

In [65]:
gbc = GradientBoostingClassifier(n_estimators=500)
plot_confusion_matrix(gbc,all_data_encoded[0:11094], target)

Predicted duration  10 minutes  120 minutes  30 minutes  45 minutes  \
Actual duration                                                       
10 minutes                   0            0           1           0   
120 minutes                  0            3           2           2   
20 minutes                   0            0           6           1   
30 minutes                   1            2        1658         166   
45 minutes                   0            4         286         333   
65 minutes                   0            2          94          96   
80 minutes                   0            0           4           2   

Predicted duration  65 minutes  80 minutes  
Actual duration                             
10 minutes                   0           0  
120 minutes                  3           0  
20 minutes                   0           0  
30 minutes                  18           0  
45 minutes                  34           0  
65 minutes                  54           1  


In [67]:
lgbc = lgbm.LGBMClassifier(num_leaves=100,    learning_rate=0.15,
    n_estimators=900)

In [68]:
#great
plot_confusion_matrix(lgbc,all_data_encoded[0:11094], target)

Predicted duration  120 minutes  30 minutes  45 minutes  65 minutes  \
Actual duration                                                       
10 minutes                    0           1           0           0   
120 minutes                   4           2           4           0   
20 minutes                    0           4           3           0   
30 minutes                    2        1646         165          32   
45 minutes                    2         226         399          30   
65 minutes                    0          67          80         100   
80 minutes                    0           3           1           0   

Predicted duration  80 minutes  
Actual duration                 
10 minutes                   0  
120 minutes                  0  
20 minutes                   0  
30 minutes                   0  
45 minutes                   0  
65 minutes                   0  
80 minutes                   3  


In [72]:
#okayish
plot_confusion_matrix(bgc,all_data_encoded[0:11094], target)

Predicted duration  120 minutes  30 minutes  45 minutes  65 minutes  \
Actual duration                                                       
10 minutes                    0           1           0           0   
120 minutes                   2           2           4           2   
20 minutes                    0           6           1           0   
30 minutes                    2        1652         164          27   
45 minutes                    0         280         343          34   
65 minutes                    3          88          81          75   
80 minutes                    0           1           4           0   

Predicted duration  80 minutes  
Actual duration                 
10 minutes                   0  
120 minutes                  0  
20 minutes                   0  
30 minutes                   0  
45 minutes                   0  
65 minutes                   0  
80 minutes                   2  


In [278]:
pd.DataFrame({'Delivery_Time':np.vectorize(reversefactor.get)(lgbc.predict(all_data_encoded[11094:]))}).to_excel(
    "Submissions/Second_submission_lgbc_est_900_lr_15.xlsx",index=False)

In [69]:
abc = AdaBoostClassifier()
etc = ExtraTreesClassifier()
bgc = BaggingClassifier()


In [70]:
plot_confusion_matrix(abc,all_data_encoded[0:11094], target)

Predicted duration  30 minutes  45 minutes  80 minutes
Actual duration                                       
10 minutes                   1           0           0
120 minutes                  2           2           6
20 minutes                   7           0           0
30 minutes                1785          46          14
45 minutes                 544         106           7
65 minutes                 184          54           9
80 minutes                   4           3           0


In [71]:
plot_confusion_matrix(etc,all_data_encoded[0:11094], target)

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Predicted duration  120 minutes  30 minutes  45 minutes  65 minutes  \
Actual duration                                                       
10 minutes                    0           1           0           0   
120 minutes                   2           4           3           1   
20 minutes                    0           5           2           0   
30 minutes                    3        1633         168          41   
45 minutes                    0         362         262          33   
65 minutes                    2         125          58          62   
80 minutes                    0           5           1           0   

Predicted duration  80 minutes  
Actual duration                 
10 minutes                   0  
120 minutes                  0  
20 minutes                   0  
30 minutes                   0  
45 minutes                   0  
65 minutes                   0  
80 minutes                   1  


In [245]:
pd.DataFrame({'Delivery_Time':np.vectorize(reversefactor.get)(lgbc.predict(all_data_encoded[11094:]))}).to_excel(
    "Submissions/Second_submission_rfc.xlsx",index=False)